<a href="https://colab.research.google.com/github/krmoos/arduino-secret-door/blob/main/ema20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from getDataFrame import *
import datetime 
import pandas_ta as ta

indexName = "nasdaq"
testNbr = "2"

#filter dataframe
fromDatetime = datetime.datetime(year=2022,month=8,day=22,hour=5,minute=5)
toDatetime = datetime.datetime(year=2022,month=8,day=22,hour=17,minute=5)

#working hours
workingFromHour = 6
workingToHour = 14

testResultsFolder = ".\\TestResults\\"+testNbr + "\\"



os.mkdir(testResultsFolder)

getDataFrame(indexName) #getData from localfiles and convert, deduplicates and save to dataframe
df = totalDataframe
df=df.dropna()
df=df.dropna(axis=0)


df["EMA100"] = ta.ema(df.Close, length=100) #add ema100 (svarende til ema20 på 5min)
df.dropna(axis=0)

#print("Len df1 " + str(len(df.index)))
df = df.loc[fromDatetime:toDatetime] #filter data

# Aggregate/convert to 5 mins candles
# df = df.groupby(pd.Grouper(freq='5Min')).agg({"Open": "first", 
#                                              "Close": "last", 
#                                              "Low": "min", 
#                                              "High": "max"})
# df.columns = ["Open", "Close", "Low", "High"]

#Above/Below EMA20
aboveBelowCt = [0] * len(df)
for row in range(0, len(df)):
    aboveBelowCt[row] = 0
    if df.Low[row]>df.EMA20[row]: #above EMA = positiv = støtte
        aboveBelowCt[row] = aboveBelowCt[row -1] + 1
    if df.High[row]<df.EMA20[row]: #below EMA = negativ = modstand
        aboveBelowCt[row] = aboveBelowCt[row -1] - 1
    if df.High[row]>df.EMA20[row] and df.Low[row]<df.EMA20[row]:
        aboveBelowCt[row] = 0
df['aboveBelowCt']=aboveBelowCt

#Working hours
#0 = not working 
#1 = working
working = [0] * len(df)
for row in range(0, len(df)):
    working[row] = 0
    if (df.index.hour[row] >= workingFromHour and df.index.hour[row] <= workingToHour):
        working[row] = 1
df['Working']=working



#TotalSignal
TotSignal = [0] * len(df)
for row in range(0, len(df)):
    TotSignal[row] = 0
    if df.aboveBelowCt[row]==0 and df.aboveBelowCt[row-1]<=-1 and df.High[row]>df.EMA20[row]: #selling-signal - touch ema after XX - high above ema
        TotSignal[row]=1 #selling-signal entry point
    if  df.aboveBelowCt[row]==0 and df.aboveBelowCt[row-1]>=1 and df.Low[row]<df.EMA20[row]:#buying-signal - touch ema after XX - low below ema
        TotSignal[row]=2

df['TotSignal']=TotSignal




import numpy as np
def pointpos(x):
    if x['TotSignal']==1:
        return x['High']+1e-3
    elif x['TotSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)


import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
#dfpl = df[:100]

dfpl = df[1:500]


#100:350
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA20, line=dict(color='orange', width=1), name="EMA20")
                ])


# fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
#                marker=dict(size=10, color="MediumPurple"),
#                name="Signal")
fig.show()
fig.write_html(testResultsFolder + 'chart.html', auto_open=True)


#dfpl = df[:100] #OBS!!!



def SIGNAL():
    return dfpl.TotSignal


from backtesting import Strategy
import talib
#from backtesting.test import SMA

def EMA_Backtesting(values, n):
    """
    Return exponential moving average of `values`, at
    each step taking into account `n` previous values.
    """
    close = pd.Series(values)
    return talib.EMA(close, timeperiod=n)

class EmaStrat(Strategy):
    initsize = 0.05
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.ema1 = self.I(EMA_Backtesting, self.data.Close, 100)
        #self.ema1 = self.I(EMA, close, 20)

    def next(self):
        super().next()
        
        #if(self.signal1>0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0):
        #    self.mysize=self.mysize*2
        #elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
        #    self.mysize=self.initsize

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Close[-1] - 10#10e-4
            tp1 = self.data.Close[-1] + 6#5e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + 10#10e-4
            tp1 = self.data.Close[-1] - 6#5e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)


from backtesting import Backtest

bt = Backtest(dfpl, EmaStrat, cash=1000000, margin=25/25000, commission=.00)
print("Len df " + str(len(df.index)))
print("Len dfpl " + str(len(dfpl.index)))
stat = bt.run()


#stat.tail()
path = r"" + testResultsFolder + indexName +'-stats.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
stat['_trades'].to_excel(writer, sheet_name = 'trades')
stat.to_excel(writer, sheet_name = 'stats')
stat['_equity_curve'].to_excel(writer, sheet_name = 'equity_curve')
writer.save()
writer.close()


#stat['_trades'].to_excel(r"" + indexName +'.xlsx', index = False, sheet_name = "trades")
#stat.to_excel(r"" + indexName +'.xlsx', index = False, sheet_name = "stats")
#stat

#stat['_equity_curve']

bt.plot(resample=True)



ModuleNotFoundError: ignored

# Ny sektion

# Ny sektion